# Introducción al Market Basket Analysis

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

sns.set()

### Lectura de datos

In [9]:
groceries = pd.read_csv('Data1/Groceries_dataset.csv',delimiter=',')

In [10]:
groceries.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [11]:
groceries.shape

(38765, 3)

In [19]:
groceries.columns = ['id_cliente','fecha','producto']

In [20]:
groceries.head()

,id_cliente,fecha,producto
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


### Producto más frecuentes

¿cuántos productos hay?


In [21]:
groceries['producto'].nunique()

167

In [23]:
groceries['id_cliente'].munique() # transformar a object id de cliente 

AttributeError: 'Series' object has no attribute 'munique'

In [16]:
groceries['producto'].value_counts()

whole milk               2502
other vegetables         1898
rolls/buns               1716
soda                     1514
yogurt                   1334
                         ... 
rubbing alcohol             5
bags                        4
baby cosmetics              3
kitchen utensil             1
preservation products       1
Name: producto, Length: 167, dtype: int64

In [24]:
groceries['producto'].value_counts().head(20)

whole milk            2502
other vegetables      1898
rolls/buns            1716
soda                  1514
yogurt                1334
root vegetables       1071
tropical fruit        1032
bottled water          933
sausage                924
citrus fruit           812
pastry                 785
pip fruit              744
shopping bags          731
canned beer            717
bottled beer           687
whipped/sour cream     662
newspapers             596
frankfurter            580
brown bread            571
pork                   566
Name: producto, dtype: int64

## Preparación del conjunto de datos para las reglas de asociación

In [26]:
groceries.head()

,id_cliente,fecha,producto
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [27]:
trx = groceries.groupby(['id_cliente','fecha']).agg(lista_productos = ('producto', lambda x: x.tolist()))

In [28]:
trx.head()

lista_productos
id_cliente fecha                                                        
1000       15-03-2015  [sausage, whole milk, semi-finished bread, yog...
           24-06-2014                  [whole milk, pastry, salty snack]
           24-07-2015                     [canned beer, misc. beverages]
           25-11-2015                        [sausage, hygiene articles]
           27-05-2015                         [soda, pickled vegetables]

In [29]:
trx = trx['lista_productos'].tolist()

In [30]:
trx

[['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['whole milk', 'pastry', 'salty snack'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['soda', 'pickled vegetables'],
 ['frankfurter', 'curd'],
 ['sausage', 'whole milk', 'rolls/buns'],
 ['whole milk', 'soda'],
 ['beef', 'white bread'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['frozen vegetables', 'other vegetables'],
 ['butter', 'whole milk'],
 ['tropical fruit', 'sugar'],
 ['butter milk', 'specialty chocolate'],
 ['sausage', 'rolls/buns'],
 ['root vegetables', 'detergent'],
 ['frozen meals', 'dental care'],
 ['rolls/buns', 'rolls/buns'],
 ['dish cleaner', 'cling film/bags'],
 ['canned beer', 'frozen fish'],
 ['other vegetables', 'hygiene articles'],
 ['pip fruit', 'whole milk', 'tropical fruit'],
 ['rolls/buns', 'red/blush wine', 'chocolate'],
 ['other vegetables', 'shopping bags'],
 ['whole milk', 'chocolate', 'packaged fruit/vegetables', 'rolls/buns'],
 ['root vegetables', 'whole milk'

## Reglas de asociación

In [31]:
!pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.4/1.4 MB 21.3 MB/s eta 0:00:00


In [32]:
from mlxtend.preprocessing import TransactionEncoder

In [33]:
te = TransactionEncoder()

In [34]:
transformadas = te.fit(trx).transform(trx)

In [35]:
transformadas

array([[False, False, False, ...,  True,  True, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [36]:
df = pd.DataFrame(transformadas, columns=te.columns_)
df.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [37]:
from mlxtend.frequent_patterns import apriori

frecuencias = apriori(df, min_support=0.001, use_colnames=True)

In [38]:
frecuencias.shape

(750, 2)

In [39]:
from mlxtend.frequent_patterns import association_rules

In [40]:
reglas = association_rules(frecuencias, metric = 'confidence', min_threshold=0.05, )

In [41]:
reglas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(UHT-milk),(bottled water),0.021386,0.060683,0.001069,0.050000,0.823954,-0.000228,0.988755,-0.179204
1,(UHT-milk),(other vegetables),0.021386,0.122101,0.002139,0.100000,0.818993,-0.000473,0.975443,-0.184234
2,(UHT-milk),(rolls/buns),0.021386,0.110005,0.001804,0.084375,0.767013,-0.000548,0.972009,-0.236873
3,(UHT-milk),(sausage),0.021386,0.060349,0.001136,0.053125,0.880298,-0.000154,0.992371,-0.121998
4,(UHT-milk),(soda),0.021386,0.097106,0.001270,0.059375,0.611444,-0.000807,0.959887,-0.393704
...,...,...,...,...,...,...,...,...,...,...
445,"(soda, sausage)",(whole milk),0.005948,0.157923,0.001069,0.179775,1.138374,0.000130,1.026642,0.122281
446,"(sausage, whole milk)",(soda),0.008955,0.097106,0.001069,0.119403,1.229612,0.000200,1.025320,0.188423
447,"(yogurt, whole milk)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,0.000797,1.082219,0.548014
448,"(yogurt, sausage)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,0.000563,1.131541,0.384877


In [42]:
reglas.sort_values(by = 'lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
447,"(yogurt, whole milk)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,0.000797,1.082219,0.548014
449,"(sausage, whole milk)",(yogurt),0.008955,0.085879,0.001470,0.164179,1.911760,0.000701,1.093681,0.481231
121,(specialty chocolate),(citrus fruit),0.015973,0.053131,0.001403,0.087866,1.653762,0.000555,1.038081,0.401735
448,"(yogurt, sausage)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,0.000563,1.131541,0.384877
165,(flour),(tropical fruit),0.009757,0.067767,0.001069,0.109589,1.617141,0.000408,1.046969,0.385385
...,...,...,...,...,...,...,...,...,...,...
283,(pastry),(other vegetables),0.051728,0.122101,0.003676,0.071059,0.581972,-0.002640,0.945054,-0.431003
254,(misc. beverages),(whole milk),0.015772,0.157923,0.001403,0.088983,0.563459,-0.001087,0.924327,-0.440455
10,(beef),(soda),0.033950,0.097106,0.001804,0.053150,0.547335,-0.001492,0.953576,-0.461236
406,(specialty chocolate),(whole milk),0.015973,0.157923,0.001337,0.083682,0.529892,-0.001186,0.918979,-0.474121
